# Pure State Tomography Experiments
This notebook contains code for running experiments on IBM quantum computers.

## First Time Setup
Before running the code in this repository, you will need to set up the IBM Qiskit Runtime. First, create a file named config.ini, which will contain your IBM credentials. The format is as follows:

```ini
[IBM]
token = <YOUR_IBM_TOKEN>
```

This program will read the config.ini file for your API token. Now, run the following code to save your IBM token to your machine for further use. You will no longer need to run this cell afterwards. 

In [ ]:
import configparser
from qiskit_ibm_runtime import QiskitRuntimeService

with open("config.ini", "r") as cf:
    cp = configparser.ConfigParser()
    cp.read_file(cf)
    api_token = cp.get("IBM", "token")
QiskitRuntimeService.save_account(channel="ibm_quantum", token=api_token, overwrite=True)

You will also need to install the prerequisite python packages.

In [ ]:
import sys

!{sys.executable} -m pip install qiskit
!{sys.executable} -m pip install qiskit_aer
!{sys.executable} -m pip install qiskit_ibm_provider
!{sys.executable} -m pip install qiskit_ibm_runtime
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install enum

### Running Experiments
First, import the necessary packages and set up the code:

In [ ]:
from numpy import (
    ndarray,
    reshape,
    linalg,
    set_printoptions,
    pi,
)
from qiskit_aer import AerSimulator
from pure_state_tomography import tomography
import putils
import qutils
import qiskit
from measurement_manager import measurement_manager

set_printoptions(linewidth=100)

The following function runs the experiment using the `pure_state_tomography` package. 

For runs on real IBM quantum computers (i.e. not simulator or statevector runs), the first concluded run for an input will output a job file containing the IBM job ids, which the `tomography` class will use to recall the contents of previous runs. Do note that because concurrent IBM jobs are limited to 3 per free account, you may have to run this code several times to get all of the needed measurement results.

In [ ]:
talg = tomography()

def run(
    mm: measurement_manager,
    state: ndarray | qiskit.QuantumCircuit,
    verbose: bool = True,
    job_file: str = None,
    hadamard: bool = False,
):
    putils.fprint("Running inference at {} shots\n".format(mm.n_shots))

    # reset measurement manager state
    mm.set_state(state)
    
    if type(state) is qiskit.QuantumCircuit:
        state = state.copy()
    
    # run pure state tomography
    res = talg.pure_state_tomography(
        mm=mm, verbose=verbose, job_file=job_file, hadamard=hadamard
    )

    if res is not None:        
        if type(state) is ndarray:
            if state.ndim > 1:
                res = (
                    reshape(
                        res,
                        (
                            state.shape[0],
                            state.shape[0],
                        ),
                    ).T
                    * 2
                )
            putils.fprint(
                "Reconstructed {}:\n{}".format(
                    "vector" if state.ndim == 1 else "matrix", res
                )
            )
            putils.fprint(
                "% Error: {}\n".format(100 * linalg.norm(state- res))
            )
        elif type(state) is qiskit.QuantumCircuit:
            state = qutils.circuit_to_statevector(state)
            
            if state.ndim > 1:
                res = (
                    reshape(
                        res,
                        (
                            state.shape[0],
                            state.shape[0],
                        ),
                    ).T
                    * 2
                )

            putils.fprint(
                "Original {}:\n{}".format(
                    "vector" if state.ndim == 1 else "matrix", state
                )
            )
            putils.fprint(
                "Reconstructed {}:\n{}".format(
                    "vector" if state.ndim == 1 else "matrix", res
                )
            )

            putils.fprint(
                "% Error: {}\n".format(100 * linalg.norm(state - res))
            )

The following code sets up and runs the experiment on an IBM quantum computer using our previously defined run function. Using a single measurement manager object is recommended. 

In [ ]:
VERBOSITY = True

qutils.EPSILON = 5e-3

JOB_FILE = "job_2023_11_13T_17_06_44.txt"

# put state code here
state = qiskit.QuantumCircuit(3)
state.h(0)

mm = measurement_manager(
    n_shots=16384,
    execution_type=qutils.execution_type.ibm_qpu,
    verbose=VERBOSITY,
)

run(mm=mm, state=state, verbose=VERBOSITY, job_file=JOB_FILE, hadamard=True)

The following code sets up and runs the experiment many times on local noisy simulators. 

In [17]:
%%capture
VERBOSITY = False
mm = measurement_manager(
    n_shots=16384,
    execution_type=qutils.execution_type.simulator,
    verbose=VERBOSITY,
)
for a in range(12):
    # put state code here
    state = qiskit.QuantumCircuit(3)
    state.h(0)
    run(mm=mm, state=state, verbose=VERBOSITY, hadamard=True)

KeyboardInterrupt: 